# workingBCODMO
Krista Longnecker, 2 July 2025\
Make a new notebook to work with the data, the other one is getting too confusing\
This version assumes I already have the BCO-DMO dataset in hand and makes it into the format required by CMAP.

In [48]:
#some of these are residual from assembling the data file, keep for now.
import pandas as pd
import requests
import os
import json

from datetime import datetime, timedelta, timezone
from SPARQLWrapper import SPARQLWrapper, POST, JSON
from frictionless import describe, Package

In [49]:
#use this for debugging
import pdb 
#put this where I need to stop #pdb.set_trace()

In [50]:
#read in the package that was already made (starting in the middle of the process while I sort this out)
biosscope = Package('datapackage.json')

In [51]:
#various commands I have been trying to see what is biosscope
biosscope.get_resource
biosscope.resources

[{'name': 'zooscan_images_bats_ae1614',
  'type': 'file',
  'sources': [{'path': 'http://lod.bco-dmo.org/id/dataset/853440',
               'title': 'ZooSCAN images of zooplankton collected during BATS '
                        'MOCNESS tows during R/V Atlantic Explorer cruises '
                        'AE1614, AE1712, AE1830, and AE1819 in the vicinity of '
                        'the Bermuda Atlantic Time-series Study from 2016 to '
                        '2018',
               'doi': '10.26008/1912/bco-dmo.853440.1'}],
  'path': 'https://datadocs.bco-dmo.org/data/305/Zooplankton_Gradients/853440/1/data/ZooSCAN_Images_BATS_AE1614.tar.gz',
  'scheme': 'https',
  'format': 'tar',
  'compression': 'gz',
  'encoding': 'utf-8',
  'bcodmo:parameters': []},
 {'name': 'zooscan_images_bats_ae1712',
  'type': 'file',
  'sources': [{'path': 'http://lod.bco-dmo.org/id/dataset/853440',
               'title': 'ZooSCAN images of zooplankton collected during BATS '
                        'MOCNE

In [52]:
#checking out ways to access the information in biosscope
for idx,item in enumerate(biosscope.resources):   
    justFile = item.sources[0]['doi']; #apparently I need to index this
    print(justFile)

10.26008/1912/bco-dmo.853440.1
10.26008/1912/bco-dmo.853440.1
10.26008/1912/bco-dmo.853440.1
10.26008/1912/bco-dmo.853440.1
10.26008/1912/bco-dmo.854077.1
10.26008/1912/bco-dmo.857891.1
10.26008/1912/bco-dmo.861266.1
10.26008/1912/bco-dmo.920443.1
10.26008/1912/bco-dmo.964684.1
10.26008/1912/bco-dmo.964801.1
10.26008/1912/bco-dmo.964826.1


In [54]:
out = [];
for idx,item in enumerate(biosscope.resources):
    justFile = item.name;
    #pdb.set_trace()
    out.append(justFile)
 
out #this will be a list

['zooscan_images_bats_ae1614',
 'zooscan_images_bats_ae1712',
 'zooscan_images_bats_ae1819',
 'zooscan_images_bats_ae1830',
 'zooscan_bats_biovolume',
 'zooscan_mocness_output',
 'survey_biogeochemical',
 '920443_v1_biosscope_in_situ_pump_chemical_data',
 '964684_v1_amino_acids_biosscope_2021',
 '964801_v1_pump_carbohydrates_biosscope_2021',
 '964826_v1_pump_poc_pon_biosscope']

In [53]:
for idx,item in enumerate(biosscope.resources):
    justFile = item.path;
    print(justFile)

https://datadocs.bco-dmo.org/data/305/Zooplankton_Gradients/853440/1/data/ZooSCAN_Images_BATS_AE1614.tar.gz
https://datadocs.bco-dmo.org/data/305/Zooplankton_Gradients/853440/1/data/ZooSCAN_Images_BATS_AE1712.tar.gz
https://datadocs.bco-dmo.org/data/305/Zooplankton_Gradients/853440/1/data/ZooSCAN_Images_BATS_AE1819.tar.gz
https://datadocs.bco-dmo.org/data/305/Zooplankton_Gradients/853440/1/data/ZooSCAN_Images_BATS_AE1830.tar.gz
https://datadocs.bco-dmo.org/file/3YY3gVnum7AlR0/zooscan_bats_biovolume.csv
https://datadocs.bco-dmo.org/file/6YYMlVRurEQWp5/zooscan_mocness_output.csv
https://datadocs.bco-dmo.org/file/KAAGNEBc2V61jx/survey_biogeochemical.csv
https://datadocs.bco-dmo.org/file/oAJMAWlunxyLm/920443_v1_biosscope_in_situ_pump_chemical_data.csv
https://datadocs.bco-dmo.org/file/XYG3xXqfkkjG5x/964684_v1_amino_acids_biosscope_2021.csv
https://datadocs.bco-dmo.org/file/7Dxl3PMCmEl8lX/964801_v1_pump_carbohydrates_biosscope_2021.csv
https://datadocs.bco-dmo.org/file/m7zBJ4GTjvNzDY/964826

In [ ]:
#Now actually do something with a data file

In [55]:
#start with the discrete data
index = out.index('survey_biogeochemical')
print(f"Found the discrete data at index {index}") # Output: Found at index 1
#use that to find the right filename
biosscope.resources[index]

Found at index 6


{'name': 'survey_biogeochemical',
 'type': 'table',
 'sources': [{'path': 'http://lod.bco-dmo.org/id/dataset/861266',
              'title': 'BIOS-SCOPE survey biogeochemical data as collected on '
                       'Atlantic Explorer cruises (AE1614, AE1712, AE1819, '
                       'AE1916) from 2016 through 2019',
              'doi': '10.26008/1912/bco-dmo.861266.1'}],
 'path': 'https://datadocs.bco-dmo.org/file/KAAGNEBc2V61jx/survey_biogeochemical.csv',
 'scheme': 'https',
 'format': 'csv',
 'mediatype': 'text/csv',
 'encoding': 'utf-8',
 'hash': 'sha256:95f87b132193f3ae27dc6ed34371b1ef70505e3f45b8ac298953441f5463f774',
 'bytes': 481907,
 'fields': 76,
 'rows': 1370,
 'schema': {'fields': [{'name': 'Program', 'type': 'string'},
                       {'name': 'Cruise_ID', 'type': 'string'},
                       {'name': 'Cast', 'type': 'integer'},
                       {'name': 'Niskin', 'type': 'integer'},
                       {'name': 'decy', 'type': 'number'},

In [56]:
data_url = biosscope.resources[index].path

In [57]:
#the next steps I already had hand-coded in from the CSV file

In [58]:
#super easy once I have the URL:
bcodmo = pd.read_csv(data_url,na_values = ['nd']) #now I have NaN...but they get dropped when writing the file

In [59]:
bcodmo.head()

Program Cruise_ID  Cast  Niskin       decy      ISO_DateTime_UTC  \
0  BIOSSCOPE    AE1614     1       1  2016.5207  2016-07-09T14:04:00Z   
1  BIOSSCOPE    AE1614     1       2  2016.5207  2016-07-09T14:04:00Z   
2  BIOSSCOPE    AE1614     1       3  2016.5207  2016-07-09T14:04:00Z   
3  BIOSSCOPE    AE1614     1       4  2016.5207  2016-07-09T14:04:00Z   
4  BIOSSCOPE    AE1614     1       5  2016.5207  2016-07-09T14:04:00Z   

   Latitude  Longitude   Depth  Nominal_Depth  ...  V1V2_ID  V4_18s_ID  \
0   32.1647   -64.5009  10.432             10  ...      NaN        NaN   
1   32.1647   -64.5009  10.327             10  ...      NaN        NaN   
2   32.1647   -64.5009  10.506             10  ...      NaN        NaN   
3   32.1647   -64.5009  10.174             10  ...      NaN        NaN   
4   32.1647   -64.5009  10.365             10  ...      NaN        NaN   

   Sunrise  Sunset  MLD_dens125  MLD_bvfrq  MLD_densT2      DCM  VertZone  \
0      922    2322      19.8589    19.8589     15.8872  95.3077       0.0   
1      922    2322      19.8589    19.8589     15.8872  95.3077       0.0   
2      922    2322      19.8589    19.8589     15.8872  95.3077       0.0   
3      922    2322      19.8589    19.8589     15.8872  95.3077       0.0   
4      922    2322      19.8589    19.8589     15.8872  95.3077       0.0   

   Season  
0       3  
1       3  
2       3  
3       3  
4       3  

[5 rows x 76 columns]

In [60]:
#Setup the data into the CMAP format, generating a multi-page Excel file
## data

# Required variables are time, lat, lon, depth
df = pd.DataFrame(columns=['time','lat','lon','depth'])

# time --> CMAP requirement is this: #< Format  %Y-%m-%dT%H:%M:%S,  Time-Zone:  UTC,  example: 2014-02-28T14:25:55 >
# Do this in two steps so I can check the output more easily
temp = bcodmo.copy()
temp['date'] = pd.to_datetime(temp['decy'], unit='D', origin='1970-01-01')
temp['date_cmap'] = temp['date'].dt.strftime("%Y-%m-%dT%H:%M:%S")
df['time'] = temp['date_cmap']

# lat (-90 to 90) and lon (-180 to 180); use variable names at BCO-DMO 
df['lat'] = bcodmo['Latitude']
df['lon'] = bcodmo['Longitude']  #BCO-DMO already has this as negative
df['depth'] = bcodmo['Depth']

# all remaining columns in bcodmo can be considered data (may do some trimming later for times)
bcodmo_trim = bcodmo.drop(columns=['Latitude', 'Longitude', 'Depth'])
nVariables = bcodmo_trim.shape[1] #@Python, so indexing starts with 0 (rows, 1 is the columns)
# and then add to the datafile I am assembling (essentially re-order columns
df = pd.concat([df, bcodmo_trim], axis=1)

In [61]:
#metadata about the variables
#this was the original plan, now I need to pull this from BCO-DMO

# work on the second sheet: metadata about the variables; use the CMAP dataset template to setup the dataframe so I get the column headers right
fName = 'datasetTemplate.xlsx'
sheet_name = 'vars_meta_data'
vars = pd.read_excel(fName, sheet_name=sheet_name)
cols = vars.columns.tolist()
#df2 will be the dataframe with the metadat about the variables, set it up empty here
df2 = pd.DataFrame(columns=cols,index = pd.RangeIndex(1,nVariables,1))

#Need the information from BCO-DMO to fill in the metadata about the variables.

# this is only a partial list of variables for the moment
#df2['var_short_name'] = wbVar['Header']
#df2['var_long_name'] = wbVar['Description']
#df2[,'var_sensor'] = 'need this'
#df2['var_unit'] = wbVar['Unit']
df2.loc[:,('var_spatial_res')] = 'irregular'
df2.loc[:, ('var_temporal_res')] = 'irregular'

In [62]:
#metadata about the project

# finally gather up the dataset_meta_data: might setup in a separate text file later
df3 = pd.DataFrame({
    'dataset_short_name': ['BIOSSCOPE_v1'],
    'dataset_long_name': ['BIOS-SCOPE discrete sample data'],
    'dataset_version': ['1.0'],
    'dataset_release_date': ['2025-06-25'],
    'dataset_make': ['observation'],
    'dataset_source': ['Craig Carlson, Bermuda Institute of Ocean Sciences'],
    'dataset_distributor': ['Craig Carlson, Bermuda Institute of Ocean Sciences'],
    'dataset_acknowledgement': ['We thank the BIOS-SCOPE project team and the BATS team for assistance with sample collection, processing, and analysis. The efforts of the captains, crew, and marine technicians of the R/V Atlantic Explorer are a key aspect of the success of this project. This work supported by funding from the Simons Foundation International.'],
    'dataset_history': [''],
    'dataset_description': ['This dataset includes analyses from Niskin bottle samples collected on R/V Atlantic Explorer cruises as part of the BIOS-SCOPE campaign in the time period from 2016 until 2025. Included are CTD data, and survey biogeochemical samples including inorganic nutrients, particulate organic carbon and nitrogen, dissolved organic carbon, dissolved organic nitrogen, total dissolved amino acids, bacterial abundance and production.'],
    'dataset_references': ['Carlson, C. A., Giovannoni, S., Liu, S., Halewood, E. (2025) BIOS-SCOPE survey biogeochemical data as collected on Atlantic Explorer cruises (AE1614, AE1712, AE1819, AE1916) from 2016 through 2019. Biological and Chemical Oceanography Data Management Office (BCO-DMO). (Version 1) Version Date 2021-10-17. doi:10.26008/1912/bco-dmo.861266.1 [25 June 2025]'],
    'climatology': [0]
    })

#get the list of cruise names from the bcodmo data file
t = pd.DataFrame(bcodmo['Cruise_ID'].unique())
t.columns = ['cruise_names']
df3 = pd.concat([df3,t],axis=1,ignore_index = True)

In [63]:
#export the result as an Excel file with three tabs
fName_CMAP = 'data/forCMAPfromBCODMOwithPython.xlsx'
dataset_names = {'data': df, 'dataset_meta_data': df3, 'vars_meta_data': df2}
with pd.ExcelWriter(fName_CMAP) as writer:
    for sheet_name, data in dataset_names.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)